In [1]:
import hmftpy as hmf
from hmftpy.plaquettes.triangular import plaq12, plaq12z, plaq9p, plaq9d, plaq3, plaq7, plaq19, test_bonds
from hmftpy.operators import mf_ops, inner_hamiltonian, periodic_hamiltonian
from hmftpy import do_hmft
from hmftpy.plot_spins import draw_tri_lattice, draw_cluster, plot_spins
from quspin.basis import spin_basis_1d
from quspin.operators import quantum_operator
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
maxit = 50

In [2]:
def ni_op(plaq, basis):
    A, B, C = plaq['n_nearest_sublattice']
    a_op = quantum_operator({'static': [['x', [[1, j] for j in A]]]}, basis=basis, check_symm=False, check_herm=False)
    b_op = quantum_operator({'static': [['x', [[-0.5, j] for j in B]],
                                        ['y', [[0.5*np.sqrt(3), j] for j in B]]]}, basis=basis, check_symm=False, check_herm=False)
    c_op = quantum_operator({'static': [['x', [[-0.5, j] for j in C]],
                                        ['y', [[-0.5*np.sqrt(3), j] for j in C]]]}, basis=basis, check_symm=False, check_herm=False)
    return a_op + b_op + c_op

def ms_ops(plaq, basis, d='x'):
    xs = plaq['x_stripes']
    ys = plaq['y_stripes']
    zs = plaq['z_stripes']
    xs_op = quantum_operator({'static': [[d, [[1, j] for j in xs[0]] + [[-1, j] for j in xs[1]]
                                         ]]}, basis=basis, check_symm=False, check_herm=False)
    ys_op = quantum_operator({'static': [[d, [[1, j] for j in ys[0]] + [[-1, j] for j in ys[1]]
                                         ]]}, basis=basis, check_symm=False, check_herm=False)
    zs_op = quantum_operator({'static': [[d, [[1, j] for j in zs[0]] + [[-1, j] for j in zs[1]]
                                         ]]}, basis=basis, check_symm=False, check_herm=False)
    return xs_op, ys_op, zs_op

In [3]:
plaqs_lst = [plaq7, plaq9p, plaq9d, plaq12z, plaq12]
bases_lst = [spin_basis_1d(plaq['L'], pauli=0) for plaq in plaqs_lst]
ops_lst = [hmf.operators.mf_ops(plaq, bases_lst[i]) for i, plaq in enumerate(plaqs_lst)]
steps = 50
names = ['7 site', '9 site fish', '9 site diamond', '12 site zigzag', '12 site triangle']
plaqs = {name: plaqs_lst[i] for i, name in enumerate(names)}
bases = {name: bases_lst[i] for i, name in enumerate(names)}
# ops = {name: ops_lst[i] for i, name in enumerate(names)}
energies = {name: np.zeros(steps) for name in names}

Ms_ops_x = {}
Ms_ops_y = {}
for name in names:
    print(name)
    plaq = plaqs[name]
    basis = bases[name]
    Ms_ops_x[name] = ms_ops(plaqs[name], bases[name], d='x')
    Ms_ops_y[name] = ms_ops(plaqs[name], bases[name], d='y')
Mxs = {name: np.zeros(steps) for name in names}
Mys = {name: np.zeros(steps) for name in names}
Mzs = {name: np.zeros(steps) for name in names}

Ni_ops = {name: ni_op(plaqs[name], bases[name]) for name in names}
Ni = {name: np.zeros(steps) for name in names}

degens = {name: np.zeros(steps) for name in names}

J2s = np.linspace(0, 1, steps)



7 site
9 site fish
9 site diamond
12 site zigzag
12 site triangle


In [6]:
for i, J2 in enumerate(tqdm(J2s)):
    interactions = {'nearest': {'xx': 1, 'yy': 1},
                    'n_nearest': {'xx': J2, 'yy': J2}}
    for n in names:
        Hi = inner_hamiltonian(plaqs[n], interactions, bases[n])
        e, v = Hi.eigsh(k=10, which='SA')
        ei = np.argmin(e)
        
        energies[n][i] = np.min(e)
        degens[n][i] = len(e[np.abs(e-np.min(e)) < 10**-10])
        v0 = v[:,ei]
        
        Ni[n][i] = np.real(np.vdot(v0, Ni_ops[n].dot(Ni_ops[n].dot(v0))))
        
        Mxs[n][i] = np.real(np.vdot(v0, Ms_ops_x[n][0].dot(Ms_ops_x[n][0].dot(v0))))
        Mys[n][i] = np.real(np.vdot(v0, Ms_ops_x[n][1].dot(Ms_ops_x[n][1].dot(v0))))
        Mzs[n][i] = np.real(np.vdot(v0, Ms_ops_x[n][2].dot(Ms_ops_x[n][2].dot(v0))))


100%|██████████| 50/50 [01:22<00:00,  1.64s/it]


In [ ]:
J2s_lrg = np.linspace(0, 1, 5)
Mxs_19 = np.zeros(5)
Mys_19 = np.zeros(5)
Mzs_19 = np.zeros(5)
Ni_19 = np.zeros(5)
energies_19 = np.zeros(5)
b19 = spin_basis_1d(19, pauli=0)

Ms_ops_x_19 = ms_ops(plaq19, b19, d='x')
Ni_op_19 = ni_op(plaq19, b19)


rn = 1
rnn = np.sqrt(3)
rnnn = 2
for i, J2 in enumerate(tqdm(J2s_lrg)):
    interactions = {'nearest': {'xx': 1, 'yy': 1},
                    'n_nearest': {'xx': J2, 'yy': J2}}
    Hi = inner_hamiltonian(plaq19, interactions, b19)
    print('Hamiltonian formed')
    e, v = Hi.eigsh(k=1, which='SA')
    print('Hamiltonian diagonalized')
    energies_19[i] = e[0]
    v0 = v[:,0]

    Ni_19[i] = np.real(np.vdot(v0, Ni_op_19.dot(Ni_op_19.dot(v0))))

    Mxs_19[i] = np.real(np.vdot(v0, Ms_ops_x_19[0].dot(Ms_ops_x_19[0].dot(v0))))
    Mys_19[i] = np.real(np.vdot(v0, Ms_ops_x_19[1].dot(Ms_ops_x_19[1].dot(v0))))
    Mzs_19[i] = np.real(np.vdot(v0, Ms_ops_x_19[2].dot(Ms_ops_x_19[2].dot(v0))))

  0%|          | 0/5 [00:00<?, ?it/s]

Hamiltonian formed


 20%|██        | 1/5 [00:26<01:45, 26.32s/it]

Hamiltonian diagonalized
Hamiltonian formed


 40%|████      | 2/5 [01:22<01:45, 35.30s/it]

Hamiltonian diagonalized
Hamiltonian formed


 60%|██████    | 3/5 [02:28<01:29, 44.63s/it]

Hamiltonian diagonalized


In [ ]:
plt.figure(figsize=(4,3), dpi=200)
for n in names:
    plt.plot(J2s, energies[n]/plaqs[n]['L'], label=n)
plt.scatter(J2s_lrg, energies_19/19, label='19 site')
plt.legend()
plt.xlabel(r'$J_2$')
plt.ylabel(r'$e$')
plt.show()

plt.figure(figsize=(4,3), dpi=200)
for n in names:
    plt.plot(J2s, np.gradient(energies[n]/plaqs[n]['L'], alphas), label=n)
plt.legend()
plt.xlabel(r'$J_2$')
plt.ylabel(r'$de/dJ_2$')

plt.figure(figsize=(4,3), dpi=200)
for n in names:
    plt.plot(J2s, Ni[n], label=n)
plt.scatter(J2s_lrg, Ni_19, label='19 site')

plt.legend()
plt.xlabel(r'$J_2$')
plt.ylabel(r'$N_I$')


In [ ]:
for n in names:
    plt.plot(J2s, Ni[n], label=r'$N_i$')
    plt.plot(J2s, Mxs[n], label=r'$M_s^x$')
    plt.plot(J2s, Mys[n], label=r'$M_s^y$')
    plt.plot(J2s, Mzs[n], label=r'$M_s^z$')
    plt.plot(J2s, degens[n], label='Degeneracy')
    plt.title(n)
    plt.xlabel(r'$J_2$')
    plt.legend()
    plt.show()
plt.plot(J2s_lrg, Ni_19, label=r'$N_i$')
plt.plot(J2s_lrg, Mxs_19, label=r'$M_s^x$')
plt.plot(J2s_lrg, Mys_19, label=r'$M_s^y$')
plt.plot(J2s_lrg, Mzs_19, label=r'$M_s^z$')
plt.title('19 site')
plt.xlabel(r'$J_2$')
plt.legend()
plt.show()